In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

In [ ]:
poly_features = PolynomialFeatures(degree=3, include_bias=False)
scaler = scaler = pickle.load(open( "standard_scaler.out", "rb" ))
rfr = pickle.load(open("optimizedRandomForestRegressor.out", "rb" ))
crfr = rfr.best_estimator_

In [ ]:
def separate(adir):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)

In [ ]:
def estimate_pitch (estimates):
    print(type(estimates))
    a = np.array([estimates])
    ap = poly_features.fit_transform(a)
    print(ap)
    aps = scaler.transform(ap)
    return crfr.predict(aps)


def find_optimal_pitch (current_collections, collection, part):
    pitch_estimates = []
    time = current_collections[collection]['boersma'][part][0]
    
    for algo in algorithms:
        if part in current_collections[collection][algo]:
            pitch_estimates.append(current_collections[collection][algo][part][1])    
            
            if len(current_collections[collection][algo][part][0]) < len(time):
                time = current_collections[collection][algo][part][0]
    
    best_estimate = np.empty(len(time))
    
    for i in range(len(time)):
        current_pitches = []
        for pitches in pitch_estimates:
            current_pitches.append(pitches[i])
        best_estimate[i] = estimate_pitch(current_pitches)
        std[i] = standard_deviation(current_pitches)
        med[i] = median(current_pitches)
        
    return (time, best_estimate, std, med)

In [ ]:
collections = {
    "GVM009": {},
    "GVM017": {},
    "GVM019": {},
    "GVM031": {},
    "GVM097": {}
}

algorithms = ['boersma', 'crepe', 'hermes', 'maddox', 'noll', 'praat']
data_dir = '/Akamai/voice/data/pitches-vuv/'

for collection in collections:
    for algo in algorithms:
        collections[collection][algo] = {}

for algorithm in os.listdir(data_dir):
    if not algorithm in algorithms:
        continue
    for collection in os.listdir(f"{data_dir}{algorithm}"):
        if collection != 'Scherbaum Mshavanadze':
            continue
        for song in os.listdir(f"{data_dir}{algorithm}/{collection}"):
            for part in os.listdir(f"{data_dir}{algorithm}/{collection}/{song}"):
                print(part)
                if part[:6] in collections:
                    if 'AHDS' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}")
                        collections[part[:6]][algorithm][part[part.index('AHDS'):part.index('AHDS')+6]] = (x, y)
                    elif 'ALRX' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}")
                        collections[part[:6]][algorithm][part[part.index('ALRX'):part.index('ALRX')+6]] = (x, y)
                    elif 'AOLS' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}")
                        collections[part[:6]][algorithm][part[part.index('AOLS'):part.index('AOLS')+6]] = (x, y)
                    elif 'VSOA' in part:
                        x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{part}")
                        collections[part[:6]][algorithm][part[part.index('VSOA'):part.index('VSOA')+6]] = (x, y)

In [ ]:
parts = ['AHDS1M', 'AHDS2M', 'AHDS3M', 'ALRX1M', 'ALRX2M', 'ALRX3M', 'VSOAX4', 'AOLS5S']

res_dir = '/Akamai/voice/data/ground-estimate-rfr/Scherbaum Mshavanadze/'

for collection in collections:
    for part in parts:
        t, estimate, cstd, cmed = find_optimal_pitch(collections, collection, part)
        if not os.path.isdir(res_dir + collection):
            os.mkdir(res_dir + collection)
            
        if not os.path.isdir(std_dir + collection):
            os.mkdir(std_dir + collection)
        
        if not os.path.isdir(med_dir + collection):
            os.mkdir(med_dir + collection)
        
        # ground estimate
        np.savetxt(res_dir + collection + '/' + part + '.txt', np.c_[t, estimate], delimiter=' ', fmt='%f')
        
        print(f"{part}:{collection} done")

In [ ]:
import pickle



a = np.array([[1, 2, 3, 4]])
ap = poly_features.fit_transform(a)
aps = scaler.transform(ap)

current_rfr = rfr.best_estimator_
print(current_rfr.predict(aps))

In [ ]:
a = np.array([[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]])
ap = poly_features.fit_transform(a)
aps = scaler.transform(ap)
crfr.predict(aps)